# OPTIMIZANDO CODIGOS PARA ALTA PERFORMANCE

O pacote `BenchmarkTools` facilita o acompanhamento de desempenho do código em Julia, fornecendo uma estrutura para escrever e executar grupos de benchmarks, bem como comparar resultados. `BenchmarkTools` foi criado para facilitar as seguintes tarefas:

+ Organizar coleções de benchmarks em conjuntos de benchmarks gerenciáveis
+ Configurar, salvar e recarregar parâmetros de referência para conveniência, precisão e consistência
+ Executar benchmarks de forma a obter previsões de desempenho razoáveis e consistentes
+ Analisar e comparar resultados para determinar se uma alteração de código causou prejuízos ou melhorias

Sintaxe Básica:
```julia
@benchmark comando/função

Out:
  BenchmarkTools.Trial: 
      samples:          1270
      evals/sample:     1
      time tolerance:   5.00%
      memory tolerance: 1.00%
      memory estimate:  7.63 mb
      allocs estimate:  3
      minimum time:     3.03 ms (0.00% GC)
      median time:      3.80 ms (0.00% GC)
      mean time:        3.93 ms (3.97% GC)
      maximum time:     6.94 ms (6.20% GC)
  
```

In [1]:
using BenchmarkTools

## TIPOS DEFINIDOS EM FUNÇÕES

Julia é uma linguagem dinamicamente tipada que ao contrário Java ou C, o programador não precisa, necessariamente, especificar o tipo de cada variável no código. Porém, os tipos são muito importantes em Julia, de tal forma que o melhor desempenho ocorre quando são utilizadas informações do tipo para todos os dados no código. Um tipo mal definido pode atrasar muito o desempenho de uma função/codigo. A melhor forma de obter um bom desempenho é associar o tipo do dado a ser trabalhado com o tipo que a função vai calcular.

No exemplo abaixo, temos uma função que soma elementos de um vetor. No primeiro teste a função `soma_vetor1` recebe um vetor de qualquer tipo e trabalha com "soma" do tipo `Float(soma = 0.0)`. No segundo teste a mesma função e vetores, porém a variável soma é do tipo Int(soma = 0).

In [2]:
function soma_vetor1(x)
    soma = 0.0        # cuidado! soma é tipo float
    for i = x
        soma =+
    end
    return soma
end

soma_vetor1 (generic function with 1 method)

In [3]:
x1 = collect(0:0.0001:5000)    # Array{Float64,1},
x2 = LinRange(0,5000,50000001) # LinRange{Float64}
x3 = 0:0.0001:5000             # StepRangeLen{Float64}

length(x1) , length(x2) , length(x3)

(50000001, 50000001, 50000001)

In [4]:
# confirmando o tipo

display(typeof(x1))
display(typeof(x2))
display(typeof(x3))

Array{Float64,1}

LinRange{Float64}

StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}

In [5]:
@benchmark soma_vetor1(x1)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     11.385 ns (0.00% GC)
  median time:      11.795 ns (0.00% GC)
  mean time:        12.306 ns (0.00% GC)
  maximum time:     61.468 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [6]:
@benchmark soma_vetor1(x2)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     13.303 ms (0.00% GC)
  median time:      13.507 ms (0.00% GC)
  mean time:        13.562 ms (0.00% GC)
  maximum time:     15.849 ms (0.00% GC)
  --------------
  samples:          369
  evals/sample:     1

In [7]:
@benchmark soma_vetor1(x3)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     13.301 ms (0.00% GC)
  median time:      13.561 ms (0.00% GC)
  mean time:        13.658 ms (0.00% GC)
  maximum time:     16.297 ms (0.00% GC)
  --------------
  samples:          367
  evals/sample:     1

In [65]:
function soma_vetorG(x)
    soma = 0.0        # cuidado! soma é tipo float
    for i = x
        soma =+
    end
    return soma
end

soma_vetorG (generic function with 1 method)

In [67]:
@benchmark soma_vetorG(collect(0:0.0001:5000))

BenchmarkTools.Trial: 
  memory estimate:  381.47 MiB
  allocs estimate:  2
  --------------
  minimum time:     198.954 ms (4.81% GC)
  median time:      203.721 ms (4.78% GC)
  mean time:        206.893 ms (5.56% GC)
  maximum time:     248.967 ms (20.86% GC)
  --------------
  samples:          25
  evals/sample:     1

In [62]:
function soma_vetorT(x::Array{Float64,1})
    soma = 0.0        # cuidado! soma é tipo float
    for i = x
        soma =+
    end
    return soma
end

soma_vetorT (generic function with 1 method)

In [68]:
@benchmark soma_vetorT(collect(0:0.0001:5000))

BenchmarkTools.Trial: 
  memory estimate:  381.47 MiB
  allocs estimate:  2
  --------------
  minimum time:     196.232 ms (4.82% GC)
  median time:      199.103 ms (4.78% GC)
  mean time:        201.213 ms (5.60% GC)
  maximum time:     245.693 ms (22.08% GC)
  --------------
  samples:          25
  evals/sample:     1

**A função soma_vetor modificada**

Note que a variável "soma" agora é definido como `soma = soma + i`. Só esta construção já é suficiente para reduzir o desempelho do código. 

obs: O código pode ficar pior se `soma = 0` (tipo inteiro).

In [8]:
function soma_vetor2(x)
    soma = 0.0          # cuidado! soma é tipo float
    for i in x
        soma = soma + i # só este datalhe ja é suficiente para reduzir o desempenho
    end
    return soma
end

soma_vetor2 (generic function with 1 method)

In [9]:
@benchmark soma_vetor2(x1)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     56.218 ms (0.00% GC)
  median time:      56.978 ms (0.00% GC)
  mean time:        57.373 ms (0.00% GC)
  maximum time:     61.862 ms (0.00% GC)
  --------------
  samples:          88
  evals/sample:     1

In [10]:
@benchmark soma_vetor2(x2)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     62.247 ms (0.00% GC)
  median time:      63.176 ms (0.00% GC)
  mean time:        63.686 ms (0.00% GC)
  maximum time:     69.015 ms (0.00% GC)
  --------------
  samples:          79
  evals/sample:     1

In [11]:
@benchmark soma_vetor2(x3)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     101.034 ms (0.00% GC)
  median time:      102.842 ms (0.00% GC)
  mean time:        103.965 ms (0.00% GC)
  maximum time:     120.240 ms (0.00% GC)
  --------------
  samples:          49
  evals/sample:     1

O tipo Array{Float64,1} definido pelo ` collect` apresentou melhor desempenho de tempo. Veja que todos os testes apresentaram resultados muito ruins em função da construção `soma = soma + i`.

## NÃO USE VARIÁVEIS GLOBAIS EM FUNÇÕES

Variáveis globais são uma tentação na vida de qualquer programador. Porém...

In [12]:
using BenchmarkTools

In [14]:
# função calcula a soma a partir de uma variável global x

function soma_vetor_Global()
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma =+
    end
    return soma
end

soma_vetor_Global (generic function with 1 method)

In [15]:
x = collect(0:0.0001:5000);

In [16]:
@benchmark soma_vetor_Global()

BenchmarkTools.Trial: 
  memory estimate:  2.98 GiB
  allocs estimate:  149999493
  --------------
  minimum time:     2.499 s (6.10% GC)
  median time:      2.505 s (6.11% GC)
  mean time:        2.505 s (6.11% GC)
  maximum time:     2.511 s (6.13% GC)
  --------------
  samples:          2
  evals/sample:     1

A mesma função só que usando variável local passada para função

In [17]:
# função calcula a soma a partir de uma variável local X passada como valor

function soma_vetor_Local(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma =+
    end
    return soma
end

soma_vetor_Local (generic function with 1 method)

In [56]:
@benchmark soma_vetor_Local(collect(0:0.0001:5000))

BenchmarkTools.Trial: 
  memory estimate:  381.47 MiB
  allocs estimate:  2
  --------------
  minimum time:     199.364 ms (4.74% GC)
  median time:      207.686 ms (4.61% GC)
  mean time:        208.911 ms (5.50% GC)
  maximum time:     246.763 ms (21.83% GC)
  --------------
  samples:          24
  evals/sample:     1

Diferença absurda de tempo de alocação de memória entre o primeiro caso e segundo caso. 

## UTILIZE FUNÇÕES PARA SUBSTITUIR BLOCOS DE CÓDIGO

In [21]:
using BenchmarkTools

Bloco de código que utiliza variáveis globais para calcular a soma todos os elementos de um vetor

In [57]:
@benchmark begin
x = collect(0:0.0001:5000)

soma = 0.0             # cuidado! soma é tipo float
for i in x1
    soma =+
end

display(soma)
    
end

+ (generic function with 170 methods)

+ (generic function with 170 methods)

+ (generic function with 170 methods)

+ (generic function with 170 methods)

+ (generic function with 170 methods)

+ (generic function with 170 methods)

+ (generic function with 170 methods)

BenchmarkTools.Trial: 
  memory estimate:  3.35 GiB
  allocs estimate:  150000116
  --------------
  minimum time:     2.966 s (9.46% GC)
  median time:      3.195 s (9.23% GC)
  mean time:        3.195 s (9.23% GC)
  maximum time:     3.423 s (9.02% GC)
  --------------
  samples:          2
  evals/sample:     1

Função que substitui o procedimento acima e usa variáveis locais

In [30]:
# função calcula a soma a partir de uma variável local X passada como valor

function soma_vetorL(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma =+
    end
    return soma
end

soma_vetorL (generic function with 1 method)

In [59]:
@benchmark soma_vetorL(collect(0:0.0001:5000))

BenchmarkTools.Trial: 
  memory estimate:  381.47 MiB
  allocs estimate:  2
  --------------
  minimum time:     196.840 ms (4.81% GC)
  median time:      198.905 ms (4.72% GC)
  mean time:        201.207 ms (5.53% GC)
  maximum time:     245.111 ms (20.95% GC)
  --------------
  samples:          25
  evals/sample:     1

Veja que a diferença de tempo é absurda!

In [60]:
GC.gc()

## CRIAÇÃO DE MATRIZES

Quando for criar uma matriz a partir de uma função, use List Compreehsion ao invés de laços `for`, além de deixar mais claro o código é bem mais rápido.

In [17]:
using BenchmarkTools

In [33]:
# função for
function matriz_for(n)
    matriz  = ones(n,n)
    for i = 1:n
        for j = 1:n
            matriz[i,j] = 0.5*i^2 + 0.3j^2
        end
    end
    return matriz
end

matriz_for (generic function with 1 method)

In [34]:
@benchmark matriz_for(10000)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.007 s (2.46% GC)
  median time:      1.013 s (2.44% GC)
  mean time:        1.025 s (2.69% GC)
  maximum time:     1.056 s (3.49% GC)
  --------------
  samples:          5
  evals/sample:     1

In [40]:
matriz  = nothing

In [41]:
# use o garbage collector para remover o lixo da memória
GC.gc()
GC.gc()

In [42]:
# função LC
function matriz_lc(n)
   return [0.5*i^2 + 0.3*j^2 for i = 1:n, j = 1:n]
end

matriz_lc (generic function with 1 method)

In [43]:
@benchmark matriz_lc(10000)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     320.497 ms (2.17% GC)
  median time:      332.733 ms (5.29% GC)
  mean time:        335.638 ms (6.21% GC)
  maximum time:     366.133 ms (14.30% GC)
  --------------
  samples:          15
  evals/sample:     1

In [44]:
matriz  = nothing

In [45]:
# use o garbage collector para remover o lixo da memória
GC.gc()
GC.gc()

## QUAL A MELHOR: FUNÇÕES GENÉRICAS, FUNÇÕES ANÔNIMAS OU EXPRESÕES SIMBÓLICAS?

Uma dúvida que ocorre devido as possibilidades de escolhas é qual função apresenta melhor desenpenho de cálculo. As funções anônimas foram otimizadas na versão 0.5 e não há necessidade do pacote `FastAnonymous` como era nas versões anteriores.

* **Funções Genéricas**

|Vantagens   |Desvantagens |
|------------|-------------|
|Fáceis de escrever e bem próximas da escrita de uma função da matemática |Só permitem cálculo de funções definidas|
|Rápidas |A redefinição pode ser um problema|

* **Expressões Simbólicas**

|Vantagens   |Desvantagens |
|------------|-------------|
|Permitem o cálculo simbólico entre variáveis|Muito lentas em relação as funções genéricas e anônimas        |
|Podem ser utilizadas em conjunto com funções genéricas e variáveis simbólicas|Não são funções de acordo com a definição de função|

* **Funções Anônimas**

|Vantagens   |Desvantagens |
|------------|-------------|
|Velocidade praticamente iqual às funções genéricas|afdasdfa|
|asdfasdf|afdasdfa|

In [46]:
using BenchmarkTools

In [47]:
# Genérica
f_gen(x , y) = x^2 + y^3 + x^5 + y^7 + x^9 + y^11

f_gen (generic function with 1 method)

In [48]:
using SymPy
@syms x y

(x, y)

In [55]:
# Simbólica
exp_simb = x^2 + y^3 + x^5 + y^7 + x^9 + y^11

 9    5    2    11    7    3
x  + x  + x  + y   + y  + y 

In [50]:
x_vetor = rand(100, 100)
y_vetor = rand(100, 100);

In [51]:
# Função Genérica
@benchmark f_gen.(x_vetor,y_vetor)

BenchmarkTools.Trial: 
  memory estimate:  78.25 KiB
  allocs estimate:  4
  --------------
  minimum time:     2.097 ms (0.00% GC)
  median time:      2.172 ms (0.00% GC)
  mean time:        2.190 ms (0.12% GC)
  maximum time:     3.372 ms (0.00% GC)
  --------------
  samples:          2282
  evals/sample:     1

In [54]:
# Expressão Simbólica
@benchmark exp_simb.(x_vetor,y_vetor)

BenchmarkTools.Trial: 
  memory estimate:  33.49 MiB
  allocs estimate:  1060011
  --------------
  minimum time:     13.929 s (0.05% GC)
  median time:      13.929 s (0.05% GC)
  mean time:        13.929 s (0.05% GC)
  maximum time:     13.929 s (0.05% GC)
  --------------
  samples:          1
  evals/sample:     1

In [53]:
# Função Anônima
@benchmark ((x,y) -> x^2 + y^3 + x^5 + y^7 + x^9 + y^11).(x_vetor,y_vetor)

BenchmarkTools.Trial: 
  memory estimate:  78.25 KiB
  allocs estimate:  4
  --------------
  minimum time:     2.097 ms (0.00% GC)
  median time:      2.205 ms (0.00% GC)
  mean time:        2.234 ms (0.10% GC)
  maximum time:     4.127 ms (18.19% GC)
  --------------
  samples:          2237
  evals/sample:     1

## OTIMIZANDO UBUNTU E DERIVADOS

O Ubuntu e outros sistemas linux, possuem uma configuração de **"Concurrency"** que permite habilitar todos os núcleos, isso faz com que o Boot do seu sistema seja mais rápido que o normal e pode melhorar o desempenho do processamento numérico da Julia. Para habilitar o recurso, digite no shell:

```bash
sudo gedit /etc/init.d/rc
```
Para evitar problemas, salve o arquivo original como **rc.back**, caso ocorra algum erro ou instabilidade do sistema. Procure as linhas **CONCURRENCY=makefile** e descomente, e comente a linha **#CONCURRENCY=none**. Veja o código abaixo:

    # Check if we are able to use make like booting.  It require the
    # insserv package to be enabled. Boot concurrency also requires
    # startpar to be installed.
    #
    CONCURRENCY=makefile # descomente a linha
    # disable startpar, incompatible with "task" upstart jobs
    #CONCURRENCY=none   # comente a linha
    
Reinicie e veja a diferença :)